![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.


## Spark Streaming

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
import pyspark.streaming

# Lokalny kontekst Spark Streaming z 2 workerami agregujący dane z 1 sekundy.
sc = pyspark.SparkContext(master="local[2]", appName="AccessLogStreamAnalysis")
ssc = pyspark.streaming.StreamingContext(sc, 3)

In [3]:
# Czytamy z tekstowego socketa 8899
lines = ssc.socketTextStream("localhost", 8899)

In [4]:
# Funkcja pomocnicza do czytania wierszy
def get_row(line):
    cols = line.split()
    row = dict()
    source = cols[0]
    timestamp = cols[3]
    method = cols[5].replace('"', '')
    return (source, (method, timestamp))

In [5]:
# Czytanie wierszy
rows = lines.map(lambda l: get_row(l))

In [6]:
# Liczenie wiersz
count = rows.count()
count.pprint()

In [7]:
def get_sets(values):
    d = {'method': {v[0] for v in values},
        'timestamp': {v[1] for v in values}}
    return d

In [8]:
# Znajdowanie wszystkich metod w tej porcji loga
grouped = rows.groupByKey()
grouped.mapValues(get_sets).pprint()

In [9]:
# Strumień trzeba wystartować ręcznie
ssc.start()             
# Przestajemy czekać na strumień po 10s; normalnie czekamy w nieskończoność używając tej metody
ssc.awaitTermination(10)
# Zatrzymujemy strumień po 10s
ssc.stop(True, True)

-------------------------------------------
Time: 2017-10-01 13:56:45
-------------------------------------------

-------------------------------------------
Time: 2017-10-01 13:56:45
-------------------------------------------

-------------------------------------------
Time: 2017-10-01 13:56:48
-------------------------------------------
3

-------------------------------------------
Time: 2017-10-01 13:56:48
-------------------------------------------
('wc09.mtnk.rnc.net.cable.rogers.com', {'method': {'GET'}, 'timestamp': {'[11/Mar/2004:16:13:02'}})
('lj1048.inktomisearch.com', {'method': {'GET'}, 'timestamp': {'[09/Mar/2004:21:51:09'}})
('10.0.0.153', {'method': {'GET'}, 'timestamp': {'[10/Mar/2004:12:07:14'}})

-------------------------------------------
Time: 2017-10-01 13:56:51
-------------------------------------------
3

-------------------------------------------
Time: 2017-10-01 13:56:51
-------------------------------------------
('lj1024.inktomisearch.com', {'method': {

## Zadania

* Zbadaj dane.
* Wyczytaj inne wartości takie jak znacznik czasu czy ścieżkę.
* Policz ilość rekordów dla jednego źródła.
* ★ Policz metody dla jednego źródła.
* ★ Policz rekordy dla 5s.

Podpowiedź: dane ze strumienia są w pliku `data/access_log`.